# QAOA Braket

In [ ]:
from braket.ahs.hamiltonian import Hamiltonian
from braket.ahs.atom_arrangement import AtomArrangement
from braket.ahs.analog_hamiltonian_simulation import AnalogHamiltonianSimulation
from braket.timings import TimeSeries
from braket.ahs.driving_field import DrivingField
from braket.devices import LocalSimulator
from braket.aws import AwsDevice
from braket.aws import  AwsQuantumTask

from quera_ahs_utils.plotting import (
    show_global_drive,
    show_register
)


import numpy as np
import networkx as nx
from collections import defaultdict
import json


In [50]:
def cost_function(x, G):
    obj = 0
    for i, j in G.edges():
        if x[i] + x[j] == "11":
            obj += 2
    return - x.count("1") + obj

In [103]:
backends = {}
backends["quera_aquila"] = AwsDevice("arn:aws:braket:us-east-1::device/qpu/quera/Aquila")
backends["braket_emulator"] = LocalSimulator("braket_ahs")

In [82]:
# Prepare Schedule (Independent of the problem size)

with open("./Data/opt_schedule.json", "r") as file:
    schedule = json.load(file)

omegas = [0,0] + [i for i in schedule["omega_list"] for _ in range(2)] + [0]
deltas = [0,0] + [i for i in schedule["delta_list"] for _ in range(2)] + [0]
phis = len(omegas) * [0]
num_pulses = len(schedule["omega_list"])

t_tot = round(2.0e-6,9)
t_q = round(0.052e-6,9)
t_p = t_tot/num_pulses-t_q
t_delay = round(1.95e-6,9)
times = [0 , t_delay]
t_i = t_delay
for i in range(0, num_pulses):
    t_i += t_q
    times.append(round(t_i,9))
    t_i += t_p
    times.append(round(t_p,9))
times.append(round(t_i+t_p,9))

Omega = TimeSeries.from_lists(times, omegas)
Delta = TimeSeries.from_lists(times, deltas)
phi = TimeSeries.from_lists(times, phis)

drive = DrivingField(
    amplitude=Omega,
    phase=phi,
    detuning=Delta
)




In [ ]:
# Prepare Register

nq = 30
with open(f"./Data/problems/{nq}.json", "r") as file:
    problem = json.load(file)

G = nx.Graph()
G.add_nodes_from(range(nq))
G.add_edges_from(problem["edges"])
problem.keys()
register = AtomArrangement()
a = 6.79e-6
grid_side = problem["grid_side"]
pos = problem["pos"]
numx,numy = [grid_side,grid_side]
x_shift = a * (numx - 1) / 2
y_shift = a * (numy - 1) / 2
pos_dict = {}
for i, pos_i in enumerate(pos):
    register.add([round(pos_i[0] * a - x_shift,8), round(pos_i[1] * a - y_shift,8)])    
# show_register(register, blockade_radius = np.sqrt(2)*a)

# Prepare Hamiltonian
ahs_program = AnalogHamiltonianSimulation(
    hamiltonian=drive,
    register=register
)

# show_global_drive(drive)

In [126]:
# Run experiment
shots = 200
result = {}
# backend_name = "braket_emulator"
backend_name = "quera_aquila"
job = backends[backend_name].run(ahs_program, shots=shots)
if backend_name == "braket_emulator":
    res = job.result()
    dict_counts = defaultdict(int)
    for k, count in res.get_counts().items():
        dict_counts["".join("1" if s == "r" else "0" for s in k)] += count
    result["samples"] = dict_counts
elif backend_name == "quera_aquila":
    arn = job.id
    result['arn'] = arn
result["shots"] = shots
with open(f"./Data/{backend_name}/{nq}.json", "w") as file:
    json.dump(result, file)


ValidationException: An error occurred (ValidationException) when calling the CreateQuantumTask operation: Invalid AHS program: 1 validation error for AtomArrangementValidator
__root__
  Arrangement is too wide. Sites [Decimal('-0.00004074'), Decimal('0.00004074')] and [Decimal('0.00004074'), Decimal('-0.00003395')] have x-separation (0.00008148) bigger than 0.000075 meters (type=value_error)

In [232]:
# Postprocessing
nq = 102
# backend_name = "braket_emulator"
backend_name = "quera_aquila"
with open(f"./Data/{backend_name}/{nq}.json", "r") as file:
    result = json.load(file)

with open(f"./Data/problems/{nq}.json", "r") as file:
    problem = json.load(file)
G = nx.Graph()
G.add_nodes_from(range(nq))
G.add_edges_from(problem["edges"])

if backend_name == "quera_aquila":
    arn = result['arn']
    res = AwsQuantumTask(arn = arn).result()
    dict_counts = defaultdict(int)
    dict_counts_1 = defaultdict(int)
    defects = 0
    for k, count in res.get_counts().items():
        if not "e" in k:
            dict_counts["".join("1" if s == "r" else "0" for s in k)] += count
        else:
            defects += 1
    result["samples"] = dict_counts
cost_evals = defaultdict(int)
for k, v in result["samples"].items():
    cost = cost_function(k, G)
    cost_evals[cost] += v
result["cost"] = cost_evals 
result["defects"] = defects
result["min_cost"] = cost_function(problem["sol"], G)
print(f"Number of samples with defects: {defects}")
with open(f"./Data/{backend_name}/{nq}.json", "w") as file:
    json.dump(result, file)

Number of samples with defects: 129
